In [10]:
import torch
print(torch.cuda.is_available())

True


### 加载模型和分词器

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # torch_dtype="auto",
    # device_map="auto"
).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)

f:\anaconda\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 必做1：命名实体识别任务

### 读取ner.txt，切分成句子

In [ ]:
# 读取文件内容
# 根据空行拆分段落
words = []
sentences = []
true_labels = []    #真实类别标签 
with open("ner.txt", "r", encoding="utf-8") as f:  # 打开指定的文件
    word = []
    sentence = ""
    true_label = []
    while line := f.readline():  # 按行读取文件
        line = line.strip()  # 去掉行首行尾的空白字符
        if line:  # 如果当前行非空
            single_word, label = line.split()  # 将行中的单词和标签拆分
            word.append(single_word)
            sentence += single_word
            true_label.append(label)
        elif sentence:  # 如果遇到空行，且句子不为空
            words.append(word)
            sentences.append(sentence)
            true_labels.extend(true_label)
            word = []
            sentence = ""
            true_label = []
    if sentence:    # 处理最后一句
        words.append(word)
        sentences.append(sentence)
        true_labels.extend(true_label)
            
print(f"sentences: {len(sentences)}")
print(f"true label length: {len(true_labels)}")

sentences: 218
true label length: 13173


### prompt

In [ ]:
# 定义命名实体识别的提示模板
def create_ner_prompt(text_chunk):
    return f"""你是一个专业的古文命名实体识别专家。请仔细阅读一下古文内容，在保证理解意思的基础上，帮我做命名实体识别任务，按照如下的提取要求来执行任务：

请按照以下格式输出结果：
LOC:(此处为若干个提取出的人名，之间用'、'隔开，若没有则是空的)
LOC:(此处为若干个提取出的地点名，之间用'、'隔开，若没有则是空的)
OFI:(此处为若干个提取出的地点名，之间用'、'隔开，若没有则是空的)
BOOK:(此处为若干个提取出的地点名，之间用'、'隔开，若没有则是空的)

一个示例如下：

输入文本:
后高宗知而深歎美之。仪凤四年薨，辍朝三日，使百官以次赴宅哭之，赠开府仪同三司、并州大都督，谥曰恭。宣帝即位，授上柱国。运之为宫正也，数进谏于帝。帝不纳，反疏忌之。时运又与王轨、宇文孝伯等皆为武帝亲待。阿剌怗木儿袭职，授虎符，緫管高丽人户。

输出:
PER:高宗、恭、宣帝、运、王轨、宇文孝伯、武帝、阿剌怗木儿
LOC:并州、高丽
OFI:开府仪同三司、并州大都督、上柱国、宫正
BOOK:

现在把以下这段话作为输入，请给出输出结果：

{text_chunk}

""" 

### 输入每句话，生成prompt，运行得到每句话的预测标签
把运行成功的每句话的标签保存到日志中，若出错下次运行可从日志中恢复，并继续运行之后的句子

In [ ]:
import os

# 日志文件名
ner_journal_file = "ner_journal.txt"

# 已完成的标签列表
pred_labels = []
ner_count = 0

# 检查文件是否存在，如果不存在则创建空文件
if not os.path.exists(ner_journal_file):
    open(ner_journal_file, "w", encoding="utf-8").close()

# 读取已完成的标签
if os.path.exists(ner_journal_file):
    with open(ner_journal_file, "r", encoding="utf-8") as file:
        for line in file:
            line_list = line.strip().split(" ")  # 去掉换行符后按空格分隔
            ner_count += 1
            pred_labels.extend(line_list)
        
print(f"已从日志读取 {ner_count} 句话的标签.")

for i in range(ner_count, len(sentences)):
    print(f"正在处理第 {i+1} 段...")

    # 构建每段的 prompt
    sentence = sentences[i]
    prompt = create_ner_prompt(sentence)
    
    print("prompt OK")
    
    # 准备输入数据
    messages = [
        # {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    print("start gen")
    
    # 生成输出
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    # 处理生成的结果
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    print("response OK")
    
    # 解析response并分类实体
    entities = {"PER": [], "LOC": [], "OFI": [], "BOOK": []}

    for line in response.split("\n"):
        if line.startswith("PER:"):
            entities["PER"] = line.replace("PER:", "").strip().split("、")
        elif line.startswith("LOC:"):
            entities["LOC"] = line.replace("LOC:", "").strip().split("、")
        elif line.startswith("OFI:"):
            entities["OFI"] = line.replace("OFI:", "").strip().split("、")
        elif line.startswith("BOOK:"):
            entities["BOOK"] = line.replace("BOOK:", "").strip().split("、")
            
    print("entities OK")
            
    pred_label = ['O'] * len(sentence)
    
    # 定义标签类型
    label_types = ["PER", "LOC", "OFI", "BOOK"]

    # 遍历每种实体类型，将对应的子串在原句中找到位置，按照长度和相对位置修改标签为B、I、E、S
    for label_type in label_types:
        for entity in entities[label_type]:
            if not entity:
                continue
            start = sentence.find(entity)
            if start != -1:
                end = start + len(entity)
                if 0 <= start < len(pred_label) and 0 < end <= len(pred_label):
                    if len(entity) == 1:  # 单字实体
                        pred_label[start] = f"S-{label_type}"
                    else:  # 多字实体
                        pred_label[start] = f"B-{label_type}"  # 开始
                        pred_label[end - 1] = f"E-{label_type}"  # 结束
                        if end - start > 2:
                            pred_label[start + 1:end - 1] = [f"I-{label_type}"] * (len(entity) - 2)  # 中间
                    
    print(f"sentence length: {len(sentence)}, label length: {len(pred_label)}")
    assert len(sentence) == len(pred_label), "LABEL_LENGTH_ERROR"
    
    pred_labels.extend(pred_label)
    
    print("label OK") 
    
    # 实时写入日志
    with open(ner_journal_file, "a", encoding="utf-8") as file: # 'a' 模式表示追加写入
        file.write(" ".join(pred_label) + "\n")  # 将 list 转为空格分隔的字符串并换行

    print(f"第 {i+1} 段处理完成，结果已保存到日志.")

已从日志读取 218 句话的标签.


### ps：
为了方便自己运行debug，这里采取了保存日志并可恢复。所以上面代码的具体输出可能被覆盖了。如果有需要，可以之后提交一份产生以下f1分数的运行结果的日志文件。

### 计算f1分数

In [56]:
from sklearn.metrics import f1_score

f1 = f1_score(true_labels, pred_labels, average='macro')
print(f"f1_macro: {f1}")

f1_macro: 0.33181592684168515


### ————任务一结果分析

首先方案是把输入文件预处理成多个句子的输入文本和每个实体的真实标签。对输入做分段取适中的长度，有利于模型输出较好的结果。

经过尝试，直接通过写prompt来令大模型对各个实体打上标签的效果较差，因此选择了另一种方法：先让大模型对输入文本中的PER、LOC、OFI、BOOK这四类实体进行提取，接着通过写代码来解析处理大模型的输出。我通过prompt尝试规范模型的输出格式，方便之后处理。最后先把原句所有实体都标为O，再处理其余标签，根据分类，对每个实体，作为子串在原句字符串中查找位置，若找到，则根据子串的长度等给每个字符打上该分类的B、I、E、S。

结果很显然，这个模型的效果非常不理想，f1分数只有先前神经网络方法的一半左右。而且这个模型经常会输出一些非常奇怪的东西，常见的如输出格式不符合规范，或者凭空产生一些可能相关的词语，导致添加标签前的处理可能会去掉一部分不符合格式的输出，使分数降低。

### ————与神经网络方法的结果对比

#### 大模型方法：大模型通过预训练的生成能力完成实体提取任务，基于设计的 Prompt 生成结果。

**优势**：

- 通用性强：大模型适用于各种任务，不需要专门训练 NER 模型。
- 开发快速：只需设计 Prompt 和少量代码即可完成实验，无需大量标注数据。

**劣势**：
- 任务适配性较差：大模型在通用任务中表现优异，但对 NER 等特定任务可能缺乏微调。
- 结果不够精准：F1-macro 分数较低，表明模型未能很好地捕获实体的类别和边界。
- 生成的多样性可能导致误差：生成任务可能产生与实际需求不完全匹配的输出，增加解析难度。

#### 神经网络方法：基于神经网络直接对标注数据训练并预测实体标签。

**优势**：
- 针对性强：模型专为 NER 任务设计，通过训练数据对特定任务微调，表现更优。
- F1 分数显著更高：说明模型在识别实体类别和边界上更精准，泛化能力较好。
- 可解释性强：模型结果清晰，直接输出每个字的标签。

**劣势**：
- 依赖标注数据：需要高质量标注数据才能取得理想效果。
- 开发成本较高：涉及数据预处理、模型设计和训练，耗时较长。

&nbsp;

---

## 必做2：摘要任务

### 读取输入文件并做预处理

In [ ]:
import json

articles = []   # 新闻文本
sample_summarys = []    # 参考摘要
with open("summary.jsonl", "r", encoding="utf-8") as f:  # 打开指定的文件
    for line in f:
        # 解析 JSON 数据
        data = json.loads(line)
        article = data["article"]
        summary = data["summary"]
        temp = ""
        for sentence in article:
            temp += "".join(sentence.split())
        articles.append(temp)
        sample_summarys.append(summary)
           
print(f"articles length: {len(articles)}")
print(f"sample summarys length: {len(sample_summarys)}")

articles length: 2000
sample summarys length: 2000


### prompt模板

In [51]:
def create_summary_prompt(text_chunk):
    return f"""你是一个专业的新闻文本摘要专家。请仔细阅读输入的新闻内容，在保证理解意思的基础上，帮我做摘要任务，按照如下的提取要求来执行任务：

输入为一行文本，你在完成对输入文本的阅读以及理解后，请输出一行文字，作为该文本内容的摘要，注意，输出仅有一行，为普通文本，不要有markdown语法

现在把以下这段新闻文本作为输入，请给出输出结果：

{text_chunk}

""" 

经过尝试，摘要任务在prompt中不提供示例的效果要远好于提供示例。若是提供示例，模型会在处理很多新闻的时候错误的去处理示例，导致完全错误的输出。

### 模型处理每段新闻，生成摘要并存入日志

In [ ]:
import os
import jieba
# from rouge_score import rouge_scorer
from rouge_chinese import Rouge # 这个库比上面那个效果更好

# 日志文件名
summary_journal_file = "summary_journal.jsonl"

# 已完成的标签列表
pred_summarys = []
summary_count = 0

# 分数记录
rouge_1s = []
rouge_2s = []

# 初始化 ROUGE 计算器
# scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
rouge = Rouge()

# 检查文件是否存在，如果不存在则创建空文件
if not os.path.exists(summary_journal_file):
    open(summary_journal_file, "w", encoding="utf-8").close()

# 读取已完成的标签
if os.path.exists(summary_journal_file):
    with open(summary_journal_file, "r", encoding="utf-8") as file:
        for line in file:
            # 解析 JSON 行
            log_entry = json.loads(line)
            # 获取每一条记录中的内容
            pred_summarys.append(log_entry["pred_summary"])
            rouge_1s.append(log_entry["rouge_1"])
            rouge_2s.append(log_entry["rouge_2"])
            summary_count += 1
        
print(f"已从日志读取 {summary_count} 个新闻的摘要.")

for i in range(summary_count, len(articles)):
    print(f"正在处理第 {i+1} 段...")
    
    # 构建每段的 prompt
    article = articles[i]
    prompt = create_summary_prompt(article)
    
    print("prompt OK")
    
    messages = [
        # {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    print("start gen")
    
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print("response OK")
    
    response = response.replace("\n", "")   # 将模型输出并成一行
    pred_summary = " ".join(jieba.cut(response))    # jieba分词并用空格连成字符串
    pred_summarys.append(pred_summary)
    
    print("summary OK")
    
    # 计算 ROUGE 分数
    scores = rouge.get_scores(pred_summary, sample_summarys[i])
    rouge_1 = scores[0]['rouge-1']['r']
    rouge_2 = scores[0]['rouge-2']['r']
    # rouge_scores = scorer.score(sample_summarys[i], pred_summary)
    # rouge_1 = rouge_scores['rouge1'].recall
    # rouge_2 = rouge_scores['rouge2'].recall
    rouge_1s.append(rouge_1)
    rouge_2s.append(rouge_2)

    if (i + 1) % 100 == 0:  # 每一百个输出一下平均值
        print(f"average: ROUGE-1: {sum(rouge_1s) / (i + 1)}, ROUGE-2: {sum(rouge_2s) / (i + 1)}")
    
    # 实时写入日志
    with open(summary_journal_file, "a", encoding="utf-8") as file: # 'a' 模式表示追加写入
        log_entry = {
            "pred_summary": pred_summary,
            "rouge_1": rouge_1,
            "rouge_2": rouge_2
        }
        json.dump(log_entry, file,ensure_ascii=False)
        file.write("\n")

    print(f"第 {i+1} 段处理完成，结果已保存到日志.")

已从日志读取 1700 个新闻的摘要.
正在处理第 1701 段...
prompt OK
start gen
response OK
summary OK
第 1701 段处理完成，结果已保存到日志.
正在处理第 1702 段...
prompt OK
start gen
response OK
summary OK
第 1702 段处理完成，结果已保存到日志.
正在处理第 1703 段...
prompt OK
start gen
response OK
summary OK
第 1703 段处理完成，结果已保存到日志.
正在处理第 1704 段...
prompt OK
start gen
response OK
summary OK
第 1704 段处理完成，结果已保存到日志.
正在处理第 1705 段...
prompt OK
start gen
response OK
summary OK
第 1705 段处理完成，结果已保存到日志.
正在处理第 1706 段...
prompt OK
start gen
response OK
summary OK
第 1706 段处理完成，结果已保存到日志.
正在处理第 1707 段...
prompt OK
start gen
response OK
summary OK
第 1707 段处理完成，结果已保存到日志.
正在处理第 1708 段...
prompt OK
start gen
response OK
summary OK
第 1708 段处理完成，结果已保存到日志.
正在处理第 1709 段...
prompt OK
start gen
response OK
summary OK
第 1709 段处理完成，结果已保存到日志.
正在处理第 1710 段...
prompt OK
start gen
response OK
summary OK
第 1710 段处理完成，结果已保存到日志.
正在处理第 1711 段...
prompt OK
start gen
response OK
summary OK
第 1711 段处理完成，结果已保存到日志.
正在处理第 1712 段...
prompt OK
start gen
response OK
summary OK
第 1712 段处理完成，结果已保存到

### 计算ROUGE-1、ROUGE-2平均分数

In [57]:
print(f"average: ROUGE-1: {sum(rouge_1s) / len(articles)}, ROUGE-2: {sum(rouge_2s) / len(articles)}")

average: ROUGE-1: 0.30283124082303353, ROUGE-2: 0.09532485856554059


### ————任务二结果分析

ROUGE-1: 0.3028（30.28%）

- 含义：生成摘要中，有 30.28% 的单词出现在参考摘要中，表示模型对关键单词的捕获能力一般。
- 解释：ROUGE-1 衡量的是生成摘要和参考摘要之间的单词级匹配情况，得分偏低意味着生成摘要中漏掉了参考摘要中较多的重要词汇。

ROUGE-2: 0.0953（9.53%）

- 含义：生成摘要中，有 9.53% 的二元词组（连续的两个单词）在参考摘要中出现。
- 解释：ROUGE-2 是基于二元词组的匹配，得分低表明模型生成的摘要在保留参考摘要的句子结构或连续语义上能力较弱。

### ————案例分析

随机抽取了几个案例如下：

> {"article": ["微博 图 。", "楚天 都市报 : # 快讯 # 【 竟 有 此事 ! 襄阳 幼童 惨遭 二妈 割掉 双耳 】 今日 有 网友 爆料 : “ 在 湖北 襄阳 , 因 家里 起 了 点 小 争执 , 这个 无辜 孩子 被 自己 的 二妈 割掉 耳朵 , 下巴 也 被 砍 了 … … ”", "本报记者 求证 得知 , 案件 属实 , 男 微博 图 楚天 都市报 : # 快讯 # 【 竟 有 此事 ! 襄阳 幼童 惨遭 二妈 割掉 双耳 】 今日 有 网友 爆料 : “ 在 湖北 襄阳 , 因 家里 起 了 点 小 争执 , 这个 无辜 孩子 被 自己 的 二妈 割掉 耳朵 , 下巴 也 被 砍 了 … … ”", "本报记者 求证 得知 , 案件 属实 , 男童 今年 仅 6 岁 , 被骗 至 无人 处 割掉 双耳 , 左耳 被 丢 进 草丛 已 坏死 。", "二妈 张某 已 被 刑拘 。"], "summary": "网友 爆料 称 湖北 襄阳 一 6 岁 男童 因 家庭 争执 , 被 伯母 骗 至 无人 处 割 双耳 , 砍 下巴 , 左耳 被 丢 进 草丛 坏死 ; 伯母 张某 已 被 刑拘 。 ( 图 )", "id": 504, "label": [3, 4]}

注意到，article中为“二妈”，summary中为“伯母”，显然模型产生的生成摘要为“二妈”的可能性更大，这种参考数据的不准确也会导致评分很低。

> "summary": "[ 微山县 ] 中学生 因 走路 被 碰 了 一下 , 邀 人 暴打 校友 被 行拘 。", "id": 506, "label": [0]
> 
> {"pred_summary": "小张 因 走路 时 碰 同学 被 打伤 住院 ， 两名 施暴者 已 被 警方 训诫 。", "rouge_1": 0.29411764705882354, "rouge_2": 0.058823529411764705}

从以上这个例子可以看出，模型生成的摘要可能和参考摘要描述的事情几乎一致，但是叙述选择的中心人物不同，导致从词汇组成上来看二者差异较大。

经过简单的**案例分析**，发现效果不佳的原因可能是：

- 由于输入的文章长度可能较长，信息密度高，模型在压缩信息时遗漏了许多重要内容。
- 参考摘要可能较短，导致模型难以直接生成精确的内容匹配。
- 提供的参考摘要所使用的分词器可能与生成摘要使用的分词器不同，导致二者效果有差异，在词语接近的情况下可能分词不同导致计算rouge分数较低。
- 有些生成摘要的语意和参考摘要几乎完全相同，但模型输出可能使用了许多同义词，因此在分词的匹配比较上可能匹配度较低。

### ————提升空间和改进方向

**1. 提升信息捕获能力**
- 问题：生成摘要未能很好地覆盖参考摘要中的关键信息。
- 改进：
  - 改进模型：使用更大的预训练模型或微调在摘要任务上的模型，例如 fine-tune Qwen。
  - 调整摘要长度：允许生成更长的摘要，避免因字数限制导致信息丢失。
  - 加入关键词提取：在生成摘要时，先提取文章中的关键词，指导生成摘要包含这些重要内容。

**2. 优化生成摘要的连贯性**
- 问题：生成摘要中词语间的组合和语义流畅性较弱。
- 改进：
  - 引入高质量语料微调：使用与任务类似的领域摘要数据对模型进行微调，帮助模型生成更自然的摘要。
  - 增加后处理：通过句法分析、重新排序或填充来优化生成摘要的可读性和连贯性。

**3. 增强模型对特定领域的理解**
- 问题：模型可能对特定领域的文章（如新闻、医疗等）理解较弱。
- 改进：
  - 领域数据增强：提供更多领域相关的数据进行预训练或微调。
  - 定制 prompt：为不同领域设计专门的模板，引导模型生成更符合语境的摘要。

**4. 评价指标的多样化**
- 问题：ROUGE 分数反映了词汇匹配，但无法完全衡量语义相关性。
- 改进：
  - 引入 BERTScore 或 BLEU 等语义相关性指标，与 ROUGE 结合评估生成摘要的效果。
  - 通过人工评估生成摘要的可读性和信息覆盖率。